<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/seq2seq_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys

In [4]:
%cd drive/My\ Drive

/content/drive/My Drive


In [5]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

Using TensorFlow backend.


In [6]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['eng'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['hin'])
lines = pd.concat([X[:3000],Y[:3000]], axis=1)
print(len(lines))
lines = shuffle(lines)
lines.sample(10)

3000


,eng,hin
826,Knot increasing rapidly .,गाँठ का तेजी से बढ़ना ।
2927,Those who have been given this vaccine in them...,जिनको यह टीका लगी हुई है उनमें 99 प्रतिशत के श...
1033,Juice of pumpkin freshly taken out and 4 mint ...,ताजी लौकी का रस निकालकर पोदीना पत्ती 4 व तुलसी...
905,Rice water means the dense white water while c...,मांड यानी चावल पकाते समय बचा हुआ गाढ़ा सफेद पान...
2922,In 95 % people it gets treated in 2 to 4 weeks .,95 फीसदी लोगों में यह 2 - 4 हफ्तों में ठीक हो ...
1837,But when we look at the cerebrum we can feel t...,परन्तु जब हम सेरीब्रम को देखते हैं तो लगता है ...
2188,Though this blindness can be rectified through...,हालाँकि यह अंधापन ऑपरेशन के जरिए ठीक किया जा स...
3,"In the Extra Capsular Cataract Method , the pa...",इक्स्ट्रा कैनसूलर कैटरेक्ट एक्सट्रेक्शन विधि म...
2012,Complete riddance from HPV infection can not h...,एचपीवी संक्रमण से पूरी तरह छुटकारा तो नहीं पाय...
816,In preliminary stage the knot is painless and ...,प्रारंभिक अवस्था में गाँठ दर्दरहित होती है और ...


In [7]:
print(lines.shape)
print(lines.eng.shape)
print(lines.hin.shape)

(3000, 2)
(3000,)
(3000,)


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : 'START_ '+ x + ' _END')
  return data

In [0]:
lines.eng = process_data(lines.eng,False)
lines.hin = process_data(lines.hin,True)

In [0]:
# Lowercase all characters
# lines.eng=lines.eng.apply(lambda x: x.lower())
# lines.hin=lines.hin.apply(lambda x: x.lower())
# X.eng=X.eng.apply(lambda x: x.lower())
# Y.hin=Y.hin.apply(lambda x: x.lower())

In [0]:
# Remove quotes
# lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
# lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x))
# X.eng=X.eng.apply(lambda x: re.sub("'", '', x))
# Y.hin=Y.hin.apply(lambda x: re.sub("'", '', x))

In [0]:
# exclude = set(string.punctuation) # Set of all special characters
# # Remove all the special characters
# lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
# remove_digits = str.maketrans('', '', digits)
# lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
# lines.hin = lines.hin.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [0]:
# Remove extra spaces
# lines.eng=lines.eng.apply(lambda x: x.strip())
# lines.hin=lines.hin.apply(lambda x: x.strip())
# lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
# lines.hin=lines.hin.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
# lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [16]:
lines.sample(10)

,eng,hin
688,patient feels acute pain and burn on just touc...,START_ इसके अतिरिक्त शिर शूल कंपन तथा ज्वर भी...
2511,normally the main symptom of obesity is that ...,START_ साधारणतया मोटापे की मुख्य पहचान यह है ...
2675,there is intense headache in malaria,START_ मलेरिया में तेज सिर दर्द भी होता है । _END
65,in cancer cells multiply by itself and ignores...,START_ कैंसर में शरीर में कोशिका अपने आप ही गु...
625,chikungunia enters into human through the bite...,START_ चिकनगुनिया मानव में एडिस मच्छर के काटने...
1923,in one more context also if we want to transfe...,START_ एक और सन्दर्भ में भी कंप्यूटर में स्थान...
1187,due to the reason of it being minimal invasive...,START_ मिनिमल इनवेजिव सर्जरी होने के कारण इसमे...
2336,the patient does not pay attention to it as it...,START_ दर्द न होने के कारण मरीज उस की ओर ध्यान...
2012,complete riddance from hpv infection can not h...,START_ एचपीवी संक्रमण से पूरी तरह छुटकारा तो न...
338,urban reproductive and child health programme,START_ नगरीय प्रजनन एवं बाल स्वास्थ्य कार्यक्र...


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Hindi 
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [18]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

100

In [19]:
# Max Length of target sequence
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

103

In [25]:
print(all_eng_words)
print(all_hindi_words)

{'saliva', 'cardamom', 'sobin', 'urination', 'seeds', 'babies', 'experienced', 'dilate', 'separate', 'coriander', 'fill', 'parathas', 'decay', 'happens', 'character', 'mother', 'attractive', 'neuronal', 'wires', 'administered', 'matter', 'droplets', 'doors', 'paper', 'poverty', 'spells', 'skilful', 'may', 'unhygienic', 'digestion', 'destroyer', 'tanks', 'bark', 'save', 'constituent', 'generating', 'danger', 'ventral', 'tantrum', 'adding', 'hasty', 'historical', 'external', 'batting', 'briskly', 'detection', 'groin', 'able', 'relations', 'person', 'newborn', 'radiation', 'paddy', 'conducted', 'loss', 'links', 'handles', 'intervals', 'remain', 'devoid', 'surgeon', 'repairing', 'baby', 'respects', 'attached', 'slim', 'indians', 'effect', 'supposed', 'region', 'tip', 'morning', 'rajiv', 'spends', 'unable', 'oxidants', 'specialists', 'complain', 'attracted', 'letting', 'prerequisite', 'excess', 'acupressure', 'admitted', 'someone', 'nongovernment', 'opioid', 'better', 'consciously', 'americ

In [52]:
input_words = list(all_eng_words)
target_words = list(all_hindi_words)
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(4711, 6288)

In [53]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

6289

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [55]:
print(input_token_index)
print(target_token_index)

{'saliva': 1, 'cardamom': 2, 'sobin': 3, 'urination': 4, 'seeds': 5, 'babies': 6, 'experienced': 7, 'dilate': 8, 'separate': 9, 'coriander': 10, 'fill': 11, 'parathas': 12, 'decay': 13, 'happens': 14, 'character': 15, 'mother': 16, 'attractive': 17, 'neuronal': 18, 'wires': 19, 'administered': 20, 'matter': 21, 'droplets': 22, 'doors': 23, 'paper': 24, 'poverty': 25, 'spells': 26, 'skilful': 27, 'may': 28, 'unhygienic': 29, 'digestion': 30, 'destroyer': 31, 'tanks': 32, 'bark': 33, 'save': 34, 'constituent': 35, 'generating': 36, 'danger': 37, 'ventral': 38, 'tantrum': 39, 'adding': 40, 'hasty': 41, 'historical': 42, 'external': 43, 'batting': 44, 'briskly': 45, 'detection': 46, 'groin': 47, 'able': 48, 'relations': 49, 'person': 50, 'newborn': 51, 'radiation': 52, 'paddy': 53, 'conducted': 54, 'loss': 55, 'links': 56, 'handles': 57, 'intervals': 58, 'remain': 59, 'devoid': 60, 'surgeon': 61, 'repairing': 62, 'baby': 63, 'respects': 64, 'attached': 65, 'slim': 66, 'indians': 67, 'effec

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [57]:
print(reverse_input_char_index)
print(reverse_target_char_index)

{1: 'saliva', 2: 'cardamom', 3: 'sobin', 4: 'urination', 5: 'seeds', 6: 'babies', 7: 'experienced', 8: 'dilate', 9: 'separate', 10: 'coriander', 11: 'fill', 12: 'parathas', 13: 'decay', 14: 'happens', 15: 'character', 16: 'mother', 17: 'attractive', 18: 'neuronal', 19: 'wires', 20: 'administered', 21: 'matter', 22: 'droplets', 23: 'doors', 24: 'paper', 25: 'poverty', 26: 'spells', 27: 'skilful', 28: 'may', 29: 'unhygienic', 30: 'digestion', 31: 'destroyer', 32: 'tanks', 33: 'bark', 34: 'save', 35: 'constituent', 36: 'generating', 37: 'danger', 38: 'ventral', 39: 'tantrum', 40: 'adding', 41: 'hasty', 42: 'historical', 43: 'external', 44: 'batting', 45: 'briskly', 46: 'detection', 47: 'groin', 48: 'able', 49: 'relations', 50: 'person', 51: 'newborn', 52: 'radiation', 53: 'paddy', 54: 'conducted', 55: 'loss', 56: 'links', 57: 'handles', 58: 'intervals', 59: 'remain', 60: 'devoid', 61: 'surgeon', 62: 'repairing', 63: 'baby', 64: 'respects', 65: 'attached', 66: 'slim', 67: 'indians', 68: 'e

In [58]:
print(input_token_index)
print(reverse_input_char_index)
print(target_token_index)
print(reverse_target_char_index)

{'saliva': 1, 'cardamom': 2, 'sobin': 3, 'urination': 4, 'seeds': 5, 'babies': 6, 'experienced': 7, 'dilate': 8, 'separate': 9, 'coriander': 10, 'fill': 11, 'parathas': 12, 'decay': 13, 'happens': 14, 'character': 15, 'mother': 16, 'attractive': 17, 'neuronal': 18, 'wires': 19, 'administered': 20, 'matter': 21, 'droplets': 22, 'doors': 23, 'paper': 24, 'poverty': 25, 'spells': 26, 'skilful': 27, 'may': 28, 'unhygienic': 29, 'digestion': 30, 'destroyer': 31, 'tanks': 32, 'bark': 33, 'save': 34, 'constituent': 35, 'generating': 36, 'danger': 37, 'ventral': 38, 'tantrum': 39, 'adding': 40, 'hasty': 41, 'historical': 42, 'external': 43, 'batting': 44, 'briskly': 45, 'detection': 46, 'groin': 47, 'able': 48, 'relations': 49, 'person': 50, 'newborn': 51, 'radiation': 52, 'paddy': 53, 'conducted': 54, 'loss': 55, 'links': 56, 'handles': 57, 'intervals': 58, 'remain': 59, 'devoid': 60, 'surgeon': 61, 'repairing': 62, 'baby': 63, 'respects': 64, 'attached': 65, 'slim': 66, 'indians': 67, 'effec

In [59]:
lines = shuffle(lines)
lines.head(10)

,eng,hin
238,damage to kidney delicate filtering system pr...,START_ गुर्दों को क्षति डायबिटीज से गुर्दों मे...
2571,after all why does it happen that in similar s...,START_ आखिर ऐसा क्यों होता है कि कुछ लोग एक जै...
2257,this problem may occur in some due to mental t...,START_ कुछ में मानसिक तनाव के कारण यह समस्या ह...
1207,latin brassica campestris family cruciferae,START_ लेटिन ब्रासिका केंपेसट्रिस brassica c...
2166,one very important use of ultrasound is for th...,START_ अल्ट्रासाउंड का एक बहुत ही महत्वपूर्ण उ...
1509,acute treatment is given when seldom the pain...,START_ जब कभीकभी दर्द हो तो ’ एक्यूट ’ ट्रीटम...
1067,do not bend downwards for a while after eating...,START_ खाने के बाद कुछ देर नीचे की तरफ झुके नह...
2714,blood pressure also increases in case of disea...,START_ गुर्दे या उनकी धमनियों में बीमारी होने ...
2519,at present as the consciousness of people are ...,START_ आज जैसेजैसे लोगों में स्वस्थ जीवन तथा श...
2350,if wound appears in tumor then also getting at...,START_ अगर गाँठ में घाव बन जाए तो भी सतर्क होन...


In [35]:
# Train - Test Split
X, y = lines.eng, lines.hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
l = X_train.values.tolist()
X_train_normal = copy.deepcopy(X_train)
t = []
for item in l:
  p = item.split()
  p.reverse()
  # print(p)
  p = " ".join(p)
  t.append(p)
df = pd.DataFrame({'data':t})
X_train = df.data
X_train.shape, X_test.shape
X_train.sample(10)

1923                    summers in it in water little mix
184     constipation prevent weight much too up pick n...
280      bacterial are std maximum preeti dr to according
2066    surface white on up shows spots white spots s ...
1839                     minutes for speed slow in stroll
97      body the in sugar of amount the in increase to...
965         menses during pain bleeding excessive unusual
662     reasons materialistic to due population the ha...
758     night at bites and water clean in born get mos...
434     taken be will time much how then brain human a...
Name: data, dtype: object

In [0]:
X_train.to_pickle('X_train_eng_hin.pkl')
X_test.to_pickle('X_test_eng_hin.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
latent_dim = 400
vec_len = 100
dropout_rate = 0.2

In [39]:
# encoder_inputs = Input(shape=(None,))
# encoder_embedding = Embedding(input_dim = num_encoder_tokens, output_dim = vec_len)(encoder_inputs)
# encoder_dropout   = (TimeDistributed(Dropout(rate = dropout_rate)))(encoder_embedding)

# encoder_LSTM = CuDNNLSTM(latent_dim, return_sequences=True)(encoder_dropout)
# encoder_LSTM_2 = CuDNNLSTM(latent_dim, return_sequences=True)(encoder_LSTM)
# encoder_outputs, state_h, state_c = CuDNNLSTM(latent_dim, return_state=True)(encoder_LSTM_2)

# encoder_states = [state_h, state_c]

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None,))
# dec_emb_layer = Embedding(num_decoder_tokens,output_dim = vec_len)
# dec_emb = dec_emb_layer(decoder_inputs)

# decoder_layer_1 = CuDNNLSTM(latent_dim, return_sequences=True)(dec_emb,initial_state=encoder_states)
# decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_layer_1)
# decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

# # Define the model that will turn
# # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [41]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 20

In [70]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Epoch 1/20
43/42 [==============================] - 18s 416ms/step - loss: 7.2037 - acc: 0.1087 - val_loss: 6.2617 - val_acc: 0.1495
Epoch 2/20
43/42 [==============================] - 18s 422ms/step - loss: 5.8893 - acc: 0.1687 - val_loss: 6.0969 - val_acc: 0.1685
Epoch 3/20
43/42 [==============================] - 18s 420ms/step - loss: 5.5322 - acc: 0.1932 - val_loss: 6.0045 - val_acc: 0.1834
Epoch 4/20
43/42 [==============================] - 18s 414ms/step - loss: 5.2062 - acc: 0.2150 - val_loss: 5.9289 - val_acc: 0.1942
Epoch 5/20
43/42 [==============================] - 18s 415ms/step - loss: 4.9101 - acc: 0.2354 - val_loss: 5.9681 - val_acc: 0.1975
Epoch 6/20
43/42 [==============================] - 18s 419ms/step - loss: 4.6319 - acc: 0.2577 - val_loss: 5.8290 - val_acc: 0.2056
Epoch 7/20
43/42 [==============================] - 18s 417ms/step - loss: 4.3517 - acc: 0.2815 - val_loss: 5.8659 - val_acc: 0.2041
Epoch 8/20
43/42 [==============================] - 18s 419ms/step - 

In [0]:
model.save_weights('nmt_weights.h5')

In [0]:
model.load_weights('nmt_weights.h5')

In [73]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 400)    1884400     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 400)    2515600     input_2[0][0]                    
____________________________________________________________________________________________

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
  actual_string = copy.deepcopy(actual_string)
  predicted_string = copy.deepcopy(predicted_string)
  reference = re.split("\s",actual_string.strip())
  candidate = re.split("\s",predicted_string.strip())
  smoothie = SmoothingFunction().method4
  score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
  return score

In [0]:

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [78]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train_normal[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
print("bleu score: ", calculate_bleu_score(y_train[k:k+1].values[0][6:-4],decoded_sentence[:-4]))

Input English sentence: mental emotions are balanced and one gets rid of physical tension  restlessness and pain 
Actual Hindi Translation:  रेकी के द्वारा मानसिक भावनाओं का संतुलन होता है और शारीरिक तनाव  बैचेनी व दर्द से छुटकारा मिलता जाता है । 
Predicted Hindi Translation:  रेकी के लिए अपने भी अपने वजन का रोग ज्यादा से पता ल
bleu score:  0.19112328397109352


In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [80]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
print("bleu score: ", calculate_bleu_score(y_train[k:k+1].values[0][6:-4],decoded_sentence[:-4]))

Input English sentence: by finding the root cause treatment should be done 
Actual Hindi Translation:  मूल कारणों का पता लगा कर उपचार करना चाहिए । 
Predicted Hindi Translation:  इलाज के बाद से ही होने पर निर्भर करता है । 
bleu score:  0.1940679506664638
